In [1]:
#importing the important libraries
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import matplotlib.colors as colors
import folium
from geopy.geocoders import Nominatim
import re
import requests
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#lets start with our class toronto
class Toronto:
    def __init__(self):
        self.url = url
        self.page = page
    #check the page status
    #if the status is 200 that is OK
    #if the request status is 404 there is an Error
    #for futher case check response headers on google
    #also for any case check me on jameselieta@gmail.com
    def check_status():
        url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
        page = requests.get(url)
        status = page.status_code
        return status
    #get table from the web
    def tables(url):
        page = requests.get(url)
        soup =  BeautifulSoup(page.content,"html.parser")
        table_contents=[]
        table=soup.find('table')
        for row in table.findAll('td'):
            cell = {}
            if row.span.text=='Not assigned':
                pass
            else:
                cell['PostalCode'] = row.p.text[:3]
                cell['Borough'] = (row.span.text).split('(')[0]
                cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
                table_contents.append(cell)
        return table_contents

In [2]:
a = Toronto.check_status()
print("the response of the web page is:",a)

the response of the web page is: 200


In [3]:
new =Toronto.tables("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

In [4]:
new

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'},
 {'PostalCode': 'M4A',
  'Borough': 'North York',
  'Neighborhood': 'Victoria Village'},
 {'PostalCode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park, Harbourfront'},
 {'PostalCode': 'M6A',
  'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor, Lawrence Heights'},
 {'PostalCode': 'M7A',
  'Borough': "Queen's Park",
  'Neighborhood': 'Ontario Provincial Government'},
 {'PostalCode': 'M9A',
  'Borough': 'Etobicoke',
  'Neighborhood': 'Islington Avenue'},
 {'PostalCode': 'M1B',
  'Borough': 'Scarborough',
  'Neighborhood': 'Malvern, Rouge'},
 {'PostalCode': 'M3B',
  'Borough': 'North York',
  'Neighborhood': 'Don Mills North'},
 {'PostalCode': 'M4B',
  'Borough': 'East York',
  'Neighborhood': 'Parkview Hill, Woodbine Gardens'},
 {'PostalCode': 'M5B',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Garden District, Ryerson'},
 {'PostalCode': 'M6B', 'Borough': 'North York', 'N

In [5]:
#we have to create column names 
column_names = ['PostalCode','Borough','Neighbourhood']
new_data =  pd.DataFrame(columns=column_names)
new_data

,PostalCode,Borough,Neighbourhood


In [6]:
for data in new:
    Postal =  data['PostalCode']
    borough = data['Borough']
    neighbourhood = data['Neighborhood']
    new_data = new_data.append({'PostalCode':Postal,
                    'Borough':borough,
                     'Neighbourhood':neighbourhood
                    },ignore_index=True)

In [7]:
new_data

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"
